<a href="https://colab.research.google.com/github/hirokimituya/book_create-profitable-AI-with-Python/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Mounted at /content/drive


In [2]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=2a5bf1994ba746da9babedbc736628e8f139f81186bbb862748c5e1058ea750b
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
!pip install yahoo_finance_api2

  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3713 sha256=60adf985c14d59964b568665ff02ea065591e2611b830e8d6d7de041d14d729e
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [4]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [5]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    # url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    # r = requests.get(url)
    # with open('./data/data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [6]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [7]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [8]:
import datetime as dt
import os
import pytz

jst = pytz.timezone('Asia/Tokyo')
now = dt.datetime.now(jst)
now = now.strftime("%Y%m%d_%H%M%S")
# ディレクトリ名作成（YYYYMMDD_HHMMSS_screening）
directory = f'{now}_screening'

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [9]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4272,サンドラッグ,9989,小売業
4273,サックスバー ホールディングス,9990,小売業
4274,ジェコス,9991,卸売業
4277,グローセル,9995,卸売業


In [10]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, min_price=100, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3085]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[50749]の株価情報を取得できなかったため、スキップしました。
経過時間: 421.3441712856293秒


,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1551800,774.700012
1,Ｒｏｂｏｔ Ｈｏｍｅ,1435,建設業,705500,214.000000
2,三井松島ホールディングス,1518,鉱業,509600,2906.000000
3,ＩＮＰＥＸ,1605,鉱業,10643200,2101.000000
4,石油資源開発,1662,鉱業,527500,4925.000000
...,...,...,...,...,...
324,ＮＴＴデータグループ,9613,情報・通信業,2453000,1969.000000
325,アインホールディングス,9627,小売業,2313100,4547.000000
326,ＮＳＤ,9759,情報・通信業,740800,2636.000000
327,ヤマダホールディングス,9831,小売業,2939700,462.200012


In [11]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1551800.0,774.700012
1,ＩＮＰＥＸ,1605,鉱業,10643200.0,2101.000000
2,石油資源開発,1662,鉱業,527500.0,4925.000000
3,安藤・間,1719,建設業,773500.0,1203.000000
4,コムシスホールディングス,1721,建設業,605100.0,3134.000000
...,...,...,...,...,...
221,電源開発,9513,電気・ガス業,564000.0,2279.000000
222,東京瓦斯,9531,電気・ガス業,1073500.0,3413.000000
223,大阪瓦斯,9532,電気・ガス業,1027700.0,2350.000000
224,エイチ・アイ・エス,9603,サービス業,734900.0,1988.000000


In [12]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1551800,774.700012
1,ＩＮＰＥＸ,1605,鉱業,10643200,2101.000000
2,安藤・間,1719,建設業,773500,1203.000000
3,コムシスホールディングス,1721,建設業,605100,3134.000000
4,オリエンタル白石,1786,建設業,587900,326.000000
...,...,...,...,...,...
109,中部電力,9502,電気・ガス業,1434900,1954.000000
110,関西電力,9503,電気・ガス業,3150100,2087.000000
111,北海道電力,9509,電気・ガス業,863300,674.500000
112,電源開発,9513,電気・ガス業,564000,2279.000000


In [13]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [14]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,Ｒｏｂｏｔ Ｈｏｍｅ,1435,建設業,705500.0,214.000000
1,三井松島ホールディングス,1518,鉱業,509600.0,2906.000000
2,日本Ｍ＆Ａセンターホールディングス,2127,サービス業,5599700.0,833.599976
3,リンクアンドモチベーション,2170,サービス業,960500.0,479.000000
4,パーソルホールディングス,2181,サービス業,741400.0,2532.500000
...,...,...,...,...,...
99,レノバ,9519,電気・ガス業,561100.0,1389.000000
100,ＮＴＴデータグループ,9613,情報・通信業,2453000.0,1969.000000
101,アインホールディングス,9627,小売業,2313100.0,4547.000000
102,ＮＳＤ,9759,情報・通信業,740800.0,2636.000000


In [15]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,博報堂ＤＹホールディングス,2433,サービス業,1169200,1382.000000
1,ユーグレナ,2931,食料品,623400,805.000000
2,ＭｏｎｏｔａＲＯ,3064,小売業,1751300,1758.000000
3,ウエルシアホールディングス,3141,小売業,630700,2672.000000
4,ｇｕｍｉ,3903,情報・通信業,1685800,582.000000
5,ユーザーローカル,3984,情報・通信業,745000,1964.000000
6,三菱瓦斯化学,4182,化学,1307300,2017.000000
7,ソースネクスト,4344,情報・通信業,550200,188.000000
8,そーせいグループ,4565,医薬品,1033500,1658.000000
9,日本ペイントホールディングス,4612,化学,3831400,1148.000000


In [16]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '4_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [17]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4267,タキヒヨー,9982,卸売業
4270,蔵王産業,9986,卸売業
4275,ヤマザワ,9993,小売業
4276,やまや,9994,小売業


In [18]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 100円以上、5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_standard

'timestamp'
銘柄コード[92649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9967]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ホーブ,1382,水産・農林業,342600,1901.0
1,住石ホールディングス,1514,鉱業,672600,387.0
2,塩水港精糖,2112,食料品,289100,262.0
3,クルーズ,2138,情報・通信業,920400,1062.0
4,ｎｍｓ ホールディングス,2162,サービス業,327100,445.0
...,...,...,...,...,...
75,ビジョナリーホールディングス,9263,小売業,1077700,114.0
76,ポエック,9264,卸売業,295200,970.0
77,日本テレホン,9425,情報・通信業,285000,406.0
78,ウィルソン・ラーニング ワールドワイド,9610,サービス業,306000,155.0


In [19]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,672600.0,387.000000
1,塩水港精糖,2112,食料品,289100.0,262.000000
2,クルーズ,2138,情報・通信業,920400.0,1062.000000
3,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,335400.0,675.000000
4,出前館,2484,情報・通信業,380300.0,401.000000
5,夢みつけ隊,2673,小売業,385400.0,113.000000
6,シー・ヴイ・エス・ベイエリア,2687,小売業,293800.0,830.000000
7,アップルインターナショナル,2788,卸売業,1529900.0,548.000000
8,ポラリス・ホールディングス,3010,サービス業,268000.0,164.000000
9,ビューティ花壇,3041,卸売業,747800.0,1310.000000


In [20]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,672600,387.0
1,クルーズ,2138,情報・通信業,920400,1062.0
2,夢みつけ隊,2673,小売業,385400,113.0
3,共和レザー,3553,化学,323300,656.0
4,セプテーニ・ホールディングス,4293,サービス業,612900,423.0
5,日本山村硝子,5210,ガラス・土石製品,380400,1915.0
6,ダントーホールディングス,5337,ガラス・土石製品,341700,941.0
7,大和重工,5610,鉄鋼,415600,1450.0
8,エヌリンクス,6578,サービス業,274200,389.0
9,ミナトホールディングス,6862,電気機器,451600,777.0


In [21]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '2_東証S_UPO')

In [22]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ホーブ,1382,水産・農林業,342600.0,1901.0
1,ｎｍｓ ホールディングス,2162,サービス業,327100.0,445.0
2,クシム,2345,情報・通信業,210000.0,310.0
3,出前館,2484,情報・通信業,380300.0,401.0
4,ＳＡＮＫＯ ＭＡＲＫＥＴＩＮＧ ＦＯＯＤＳ,2762,小売業,210200.0,169.0
5,ＪＦＬＡホールディングス,3069,小売業,219600.0,182.0
6,ユナイテッド・スーパーマーケット・ホールディングス,3222,小売業,219000.0,1126.0
7,ｅｎｉｓｈ,3667,情報・通信業,331100.0,271.0
8,ディー・エル・イー,3686,情報・通信業,276400.0,257.0
9,インタートレード,3747,情報・通信業,294100.0,431.0


In [23]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ＳＡＮＫＯ ＭＡＲＫＥＴＩＮＧ ＦＯＯＤＳ,2762,小売業,210200,169.0
1,ＪＦＬＡホールディングス,3069,小売業,219600,182.0
2,リミックスポイント,3825,情報・通信業,288800,216.0
3,Ａｂａｌａｎｃｅ,3856,電気機器,2114500,4090.0
4,ゲームカード・ジョイコホールディングス,6249,機械,201300,3890.0
5,アライドテレシスホールディングス,6835,電気機器,750400,108.0
6,フェローテックホールディングス,6890,電気機器,432200,3040.0
7,住信ＳＢＩネット銀行,7163,銀行業,464100,1526.0


In [24]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '5_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [25]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

,銘柄名,コード,業種
45,ルーデン・ホールディングス,1400,建設業
46,エムビーエス,1401,建設業
55,Ｌｉｂ Ｗｏｒｋ,1431,建設業
60,フィット,1436,建設業
65,ニッソウ,1444,建設業
...,...,...,...
4091,プログリット,9560,サービス業
4092,グラッドキューブ,9561,サービス業
4093,ビジネスコーチ,9562,サービス業
4094,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [26]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_grows

'timestamp'
銘柄コード[3782]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ＩＴｂｏｏｋホールディングス,1447,建設業,428700,345.0
1,ジーエヌアイグループ,2160,医薬品,12809100,2132.0
2,地域新聞社,2164,サービス業,333600,424.0
3,ウェッジホールディングス,2388,その他金融業,531100,144.0
4,アマナ,2402,サービス業,219500,397.0
...,...,...,...,...,...
79,ｉｓｐａｃｅ,9348,サービス業,212100,1530.0
80,リニューアブル・ジャパン,9522,電気・ガス業,1994200,1011.0
81,Ｍ＆Ａ総研ホールディングス,9552,サービス業,591900,3550.0
82,マイクロアド,9553,サービス業,510200,2854.0


In [27]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,12809100.0,2132.0
1,地域新聞社,2164,サービス業,333600.0,424.0
2,ウェッジホールディングス,2388,その他金融業,531100.0,144.0
3,アマナ,2402,サービス業,219500.0,397.0
4,ベースフード,2936,食料品,1061800.0,635.0
5,タスキ,2987,不動産業,317300.0,1209.0
6,ストレージ王,2997,不動産業,2813800.0,569.0
7,パルマ,3461,不動産業,254000.0,353.0
8,アプリックス,3727,情報・通信業,303500.0,192.0
9,イメージ情報開発,3803,情報・通信業,275200.0,506.0


In [28]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ウェッジホールディングス,2388,その他金融業,531100,144.0
1,タスキ,2987,不動産業,317300,1209.0
2,ストレージ王,2997,不動産業,2813800,569.0
3,パルマ,3461,不動産業,254000,353.0
4,アプリックス,3727,情報・通信業,303500,192.0
5,イメージ情報開発,3803,情報・通信業,275200,506.0
6,アララ,4015,情報・通信業,591200,869.0
7,ＨＥＮＮＧＥ,4475,情報・通信業,935900,1040.0
8,ＢＡＳＥ,4477,情報・通信業,6868600,383.0
9,メドレックス,4586,医薬品,1326600,223.0


In [29]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '3_東証G_UPO')

In [30]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ＩＴｂｏｏｋホールディングス,1447,建設業,428700.0,345.0
1,海帆,3133,小売業,13185100.0,569.0
2,ＧＡ ｔｅｃｈｎｏｌｏｇｉｅｓ,3491,不動産業,272800.0,1383.0
3,イメージ情報開発,3803,情報・通信業,275200.0,506.0
4,Ａｉｍｉｎｇ,3911,情報・通信業,3237900.0,395.0
5,プレイド,4165,情報・通信業,282700.0,671.0
6,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,305000.0,1241.0
7,ココナラ,4176,情報・通信業,235800.0,308.0
8,エクサウィザーズ,4259,情報・通信業,300600.0,406.0
9,Ｔｒｕｅ Ｄａｔａ,4416,情報・通信業,1069500.0,509.0


In [31]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,エクサウィザーズ,4259,情報・通信業,300600,406.0
1,フリー,4478,情報・通信業,371100,2972.0
2,ステムリム,4599,医薬品,389100,822.0
3,ストリームメディアコーポレーション,4772,情報・通信業,381900,239.0
4,モンスターラボホールディングス,5255,情報・通信業,463200,328.0
5,ウェルスナビ,7342,証券、商品先物取引業,374300,1241.0
6,リファインバースグループ,7375,サービス業,356400,1335.0
7,ＢｕｙＳｅｌｌ Ｔｅｃｈｎｏｌｏｇｉｅｓ,7685,卸売業,223900,3680.0
8,ブシロード,7803,その他製品,1397400,495.0
9,メンタルヘルステクノロジーズ,9218,サービス業,212000,897.0


In [32]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [33]:
import pandas as pd

def remove_duplicate_rows(dataframe, *other_dataframes):
    """重複しているティッカーコードの行を削除する

    :param dataframe: 対象のデータフレーム
    :param other_dataframes: 重複しているかどうかの判断のためのデータフレーム
    :return 引数dataframeで重複行を削除したもの
    """
    # データフレームをコピーして新しいデータフレームを作成
    result_df = dataframe.copy()

    # 重複確認のためのセットを作成
    unique_values = set()
    for df in other_dataframes:
        unique_values.update(df['コード'])

    # 重複しているティッカーコードを削除
    result_df = result_df[~result_df['コード'].isin(unique_values)]

    return result_df

In [34]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[92649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9967]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,672600,387.000000
1,クルーズ,2138,情報・通信業,920400,1062.000000
2,アップルインターナショナル,2788,卸売業,1529900,548.000000
3,ビューティ花壇,3041,卸売業,747800,1310.000000
4,ジェイ・エスコムホールディングス,3779,情報・通信業,2380500,174.000000
5,パス,3840,情報・通信業,648800,119.000000
6,アイフリークモバイル,3845,情報・通信業,1963500,116.000000
7,Ａｂａｌａｎｃｅ,3856,電気機器,2114500,4090.000000
8,セプテーニ・ホールディングス,4293,サービス業,612900,423.000000
9,環境管理センター,4657,サービス業,1225600,695.000000


In [35]:
# ティッカーコードが重複している行を削除
filtered_tse_standard_2_remove_duplicate = remove_duplicate_rows(filtered_tse_standard_2, filtered_perfect_order_week_day_tse_standard, filtered_reverse_perfect_order_week_day_tse_standard)
filtered_tse_standard_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
2,アップルインターナショナル,2788,卸売業,1529900,548.000000
3,ビューティ花壇,3041,卸売業,747800,1310.000000
4,ジェイ・エスコムホールディングス,3779,情報・通信業,2380500,174.000000
5,パス,3840,情報・通信業,648800,119.000000
6,アイフリークモバイル,3845,情報・通信業,1963500,116.000000
9,環境管理センター,4657,サービス業,1225600,695.000000
10,ＡＩメカテック,6227,機械,748000,2915.000000
11,千代田化工建設,6366,建設業,867100,374.000000
12,ＪＡＬＣＯホールディングス,6625,不動産業,720800,223.000000
14,キョウデン,6881,電気機器,506100,598.000000


In [36]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2_remove_duplicate, '7_東証S_選抜')

In [37]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows_2 = filtered_ticker(tse_grows, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_grows_2

'timestamp'
銘柄コード[3782]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,12809100,2132.0
1,ウェッジホールディングス,2388,その他金融業,531100,144.0
2,ベースフード,2936,食料品,1061800,635.0
3,ストレージ王,2997,不動産業,2813800,569.0
4,海帆,3133,小売業,13185100,569.0
5,Ａｉｍｉｎｇ,3911,情報・通信業,3237900,395.0
6,ソーシャルワイヤー,3929,情報・通信業,679200,342.0
7,アララ,4015,情報・通信業,591200,869.0
8,ヤプリ,4168,情報・通信業,511900,1663.0
9,Ｔｒｕｅ Ｄａｔａ,4416,情報・通信業,1069500,509.0


In [38]:
# ティッカーコードが重複している行を削除
filtered_tse_grows_2_remove_duplicate = remove_duplicate_rows(filtered_tse_grows_2, filtered_perfect_order_week_day_tse_grows, filtered_reverse_perfect_order_week_day_tse_grows)
filtered_tse_grows_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,12809100,2132.0
2,ベースフード,2936,食料品,1061800,635.0
4,海帆,3133,小売業,13185100,569.0
5,Ａｉｍｉｎｇ,3911,情報・通信業,3237900,395.0
6,ソーシャルワイヤー,3929,情報・通信業,679200,342.0
8,ヤプリ,4168,情報・通信業,511900,1663.0
9,Ｔｒｕｅ Ｄａｔａ,4416,情報・通信業,1069500,509.0
10,ＪＤＳＣ,4418,情報・通信業,994500,1138.0
11,Ｃｈａｔｗｏｒｋ,4448,情報・通信業,516000,941.0
14,アンジェス,4563,医薬品,646700,101.0


In [39]:
# CSVファイル保存
save_df_as_csv(filtered_tse_grows_2_remove_duplicate, '8_東証G_選抜')